# Data Preprocessing

This code, run in Google Colaboratory was written to streamline the data preprocessing pipeline and quickly iterate on our methods of dimensionality reduction. Initially, PCA and Kernel PCA are used to for dimensionality reduction to test our hypothesis that reduced electroencephalogram data can perform comparably to raw full-dimensionality recording in deep learning models. After testing this, we intend to examine model performance on groups of raw data from various electrode placement groups. These groups are the four major lobes of the brain: frontal, temporal, parietal, and occipital.

### Filtering and Artifact Removal
Standard digital filtering and artifact removal are implemented to improve the qaulity of the signal before any principal component analysis or channel selection is performed.
>Low Pass: 5 Hz \
>High Pass: 120 Hz \
>ICA with 20 components \
>Removal of EOG and EMG artifacts \
>Standardization \

### Dimensionality Reduction Methods

PCA

Kernel PCA: \
Kernel PCA is tested with the five standard kernel types provided by Sci-kit Learn. 
1. Linear
2. RBF
3. Polynomial (3rd Order)
4. Sigmoid
5. Cosine

Grouped Electrode Selection:
1. Frontal Lobe \
2. Temporal Lobe \
3. Parietal Lobe \
4. Occipital Lobe \











## Imports

In [1]:
!pip install mne

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.6 MB 15.5 MB/s 


In [2]:
import mne
from mne.io import read_raw_edf
import numpy as np
from sklearn.decomposition import PCA, KernelPCA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
import matplotlib.pyplot as plt
from collections import OrderedDict
import pickle
import os
import os.path as op
import glob
import gc
import pickle
from tqdm import tqdm
mne.set_log_level('CRITICAL')

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Declarations

In [4]:
TRAIN_DATA_PATH     = '/content/drive/Shareddrives/BCI:FinalProject/train/'
BASELINE_DATA_PATH  = '/content/drive/Shareddrives/BCI:FinalProject/baseline/'
PCA_DATA_PATH       = '/content/drive/Shareddrives/BCI:FinalProject/pca/'
KPCA_DATA_PATH      = '/content/drive/Shareddrives/BCI:FinalProject/kpca/'
FRONTAL_DATA_PATH   = '/content/drive/Shareddrives/BCI:FinalProject/frontal/'
TEMPORAL_DATA_PATH  = '/content/drive/Shareddrives/BCI:FinalProject/temporal/'
PARIETAL_DATA_PATH  = '/content/drive/Shareddrives/BCI:FinalProject/parietal/'
OCCIPITAL_DATA_PATH = '/content/drive/Shareddrives/BCI:FinalProject/occipital/'
TEST_DATA_PATH      = '/content/drive/Shareddrives/BCI:FinalProject/test/'

In-Place Preprocessing Functions

In [5]:
def adjust_edf_import(raw):
  '''
  In-place function that adjusts the channel names and types of imported EDF data.
  '''
  cs = {}
  types = {}
  drop = ['I1', 'I2']
  for c in raw.ch_names:
      # print(c.rstrip('.').upper().replace('Z', 'z').replace('FP', 'Fp'))
      c_type = c.split(' ')[0].lower()
      name = c.split(' ')[1]
      types[name] = c_type

      if len(name) > 3:
          drop.append(name)

      cs[c] = name

  raw.rename_channels(cs)
  raw.set_channel_types(types)
  raw.set_eeg_reference(projection=True)
  raw.set_montage('standard_1005')
  return raw

def fit_apply_ica_artifact_removal(raw):
  '''
  In-place fit and apply ICA artifact removal with 20 components. Data must be 
  high pass filtered before calling this.
  '''
  ica = mne.preprocessing.ICA(n_components=20, random_state=42, max_iter=800)
  ica.fit(raw, picks=['eeg'])
  eog_arts, _ = ica.find_bads_eog(raw)
  emg_arts, _ = ica.find_bads_muscle(raw)
  ica.apply(raw, exclude=np.concatenate([eog_arts, emg_arts], axis=-1))
  del ica

Preprocess and Save Data in Drive

In [6]:
def process_raw_data( raw, 
                      standardizer=None,
                      dim_reducer=None, 
                      output_dim=None, 
                      selected_channels=None, 
                      ica=True):
  '''
  Process Raw MNE data with various selections like ICA artifact removal, 
  standardizing, dimensionality reduction, and channel selection.

  WARNING: THIS EDITS RAW IN PLACE

  Params:
  raw:                an MNE Raw object
  standardizer:       a standardizer from Sklearn
  dim_reducer:        dimensionality reduction from Sklearn
  output_dim:         output dimensionality of dim_reducer
  selected_channels:  names of EEG channels to select 
  ica:                perform ICA artifact removal?

  Returns:
  ---------
  raw: a new Raw object with processed data
  events: events extracted from the annotations in the original raw
  labels: a dictionary of int to str that gives the class label

  '''
  print('Filter and downsample...')
  raw.filter(1, 120)
  raw.resample(250)
  if ica:
    print('ICA artifact removal...')
    fit_apply_ica_artifact_removal(raw)
  
  print('Selecting all EEG Channels...')
  raw.pick_types(eeg=True)
  if selected_channels:
    print(f'Refining to only {selected_channels} channels...')
    selections = []
    for ch in raw.ch_names:
      if selected_channels in ch:
        selections.append(ch)
    print(f'Found these channels: {selections}')
    raw.pick_channels(selections)

  info = raw.info
  events, labels = mne.events_from_annotations(raw)
  labels = dict([(i, l) for i,l in enumerate(labels)])
  print('Got events...')

  if standardizer or dim_reducer:
    data = raw.get_data()
    data = data.T
    del raw
    gc.collect()

    if standardizer is not None:
      print('Standardizing...')
      data = standardizer.fit_transform(data)

    if dim_reducer is not None:
      print(f'Reducing dimensionality to {output_dim}')
      data = dim_reducer.fit_transform(data)
      info = mne.create_info(output_dim, info['sfreq'], 'eeg')

    raw = mne.io.RawArray(data.T, info)
    del data
    gc.collect()

  return raw, events, labels
  
def get_data_and_class_from_raw(raw, events):
  '''
  Chop up the MNE Raw object into individual trials based on specs used in the HGD
  paper. This uses the 4.5 second interval defined in the paper.

  Returns:
  ----------
  X_y: a list of tuples made of data and the target class (data, class)
  '''
  epochs = mne.Epochs(raw, events, tmin=-0.5, tmax=4, baseline=(-0.5, 0), preload=True)

  X_y = []
  for i in range(len(epochs)):
    X_y.append((epochs[i].get_data()[0], events[i][2] - 1))

  del epochs
  gc.collect()
  return X_y

In [7]:
def process_directory(input_dir, output_dir, standardizer=None, dim_reducer=None, n_components=None, selected_channels=None, ica=True):
  train_files = glob.glob(f'{input_dir}/*.edf') # Must download for yourself ;)
  print(f'Found {len(train_files)} available EDF files')
  for file in tqdm(train_files):
    title = os.path.split(file)[-1]
    print('\n' + file)
    raw = read_raw_edf(file).load_data()
    adjust_edf_import(raw)
    raw, events, labels = process_raw_data(raw,
                                          standardizer=standardizer,
                                          dim_reducer=dim_reducer,
                                          output_dim=n_components,
                                          selected_channels=selected_channels,
                                          ica=ica)
    X_y = get_data_and_class_from_raw(raw, events)

    pickle.dump(X_y, open(output_dir + title + '.data', 'wb'))
    del X_y
    gc.collect()

Memory Efficient Methods

In [ ]:
def process_raw_data_memory_efficient( raw, 
                      standardizer=None,
                      dim_reducer=None, 
                      output_dim=None, 
                      selected_channels=None, 
                      ica=True):
  '''
  Process Raw MNE data with various selections like ICA artifact removal, 
  standardizing, non-linear dimensionality reduction (NLDR), and channel selection. 
  This performs on each epoch of data when computing dimensionality reduction. This 
  is used for the non-linear methods, since memory usage explodes with dimensionality 
  of the data.

  We did this because our initial attempts at NLDR were leading to malloc requests
  of around 3TB, which is absolutely astronomical. 

  WARNING: THIS EDITS RAW IN PLACE

  Params:
  raw:                an MNE Raw object
  standardizer:       a standardizer from Sklearn
  dim_reducer:        dimensionality reduction from Sklearn
  output_dim:         output dimensionality of dim_reducer
  selected_channels:  names of EEG channels to select 
  ica:                perform ICA artifact removal?

  Returns:
  ---------
  X_y: a list of tuples containing data and the target like (data, target)

  '''
  print('Filter and downsample...')
  raw.filter(1, 120)
  raw.resample(250)
  if ica:
    print('ICA artifact removal...')
    fit_apply_ica_artifact_removal(raw)
  
  print('Selecting all EEG Channels...')
  raw.pick_types(eeg=True)
  if selected_channels:
    print(f'Refining to only {selected_channels} channels...')
    selections = []
    for ch in raw.ch_names:
      if selected_channels in ch:
        selections.append(ch)
    print(f'Found these channels: {selections}')
    raw.pick_channels(selections)

  info = raw.info
  events, labels = mne.events_from_annotations(raw)
  labels = dict([(i, l) for i,l in enumerate(labels)])
  print('Got events...')

  X_y = get_data_and_class_from_raw(raw, events)

  del raw
  gc.collect() 

  if standardizer or dim_reducer:
    for i, (x, y) in enumerate(X_y):
      data = x.T
      label = y
      if standardizer is not None:
        # print('Standardizing...')
        data = standardizer.fit_transform(data)

      if dim_reducer is not None:
        # print(f'Reducing dimensionality to {output_dim}')
        data = dim_reducer.fit_transform(data)
        info = mne.create_info(output_dim, info['sfreq'], 'eeg')
      X_y[i] = (data.T, y)

      del data
      gc.collect()

  return X_y

In [ ]:
def process_directory_memory_efficient(input_dir, output_dir, standardizer=None, dim_reducer=None, n_components=None, selected_channels=None, ica=True):
  train_files = glob.glob(f'{input_dir}/*.edf') # Must download for yourself ;)
  print(f'Found {len(train_files)} available EDF files')
  for file in tqdm(train_files):
    title = os.path.split(file)[-1]
    print('\n' + file)
    raw = read_raw_edf(file).load_data()
    adjust_edf_import(raw)
    X_y = process_raw_data_memory_efficient(raw,
                                          standardizer=standardizer,
                                          dim_reducer=dim_reducer,
                                          output_dim=n_components,
                                          selected_channels=selected_channels,
                                          ica=ica)

    print(X_y[0][0].shape)
    pickle.dump(X_y, open(output_dir + title + '.data', 'wb'))
    del X_y
    gc.collect()

### Create and save PCA-reduced data

In [ ]:
n_components = 30
pca = PCA(n_components=n_components)
ss = StandardScaler()
input_dir = TRAIN_DATA_PATH
output_dir = PCA_DATA_PATH + 'train/'
os.makedirs(output_dir, exist_ok = True)
process_directory(input_dir, output_dir, standardizer=ss, dim_reducer=pca, n_components=30)

In [ ]:
n_components = 30
pca = PCA(n_components=n_components)
ss = StandardScaler()
input_dir = TEST_DATA_PATH
output_dir = PCA_DATA_PATH + 'test/'
os.makedirs(output_dir, exist_ok = True)
process_directory(input_dir, output_dir, standardizer=ss, dim_reducer=pca, n_components=30)

Create and save Kernel PCA-reduced data

In [ ]:
n_components = 30
kernels = ['linear', 'poly', 'rbf', 'sigmoid', 'cosine'] 
for kernel in kernels:
  kpca = KernelPCA(n_components=n_components, kernel=kernel)
  ss = StandardScaler()
  input_dir = TRAIN_DATA_PATH
  output_dir = KPCA_DATA_PATH + kernel + '/train/'
  os.makedirs(output_dir, exist_ok = True)
  process_directory_memory_efficient(input_dir, output_dir, standardizer=ss, dim_reducer=kpca, n_components=30)

Found 14 available EDF files


  0%|          | 0/14 [00:00<?, ?it/s]


/content/drive/Shareddrives/BCI:FinalProject/train/1.edf
Filter and downsample...
ICA artifact removal...
Selecting all EEG Channels...
Got events...
(30, 1126)


  7%|▋         | 1/14 [03:26<44:40, 206.16s/it]


/content/drive/Shareddrives/BCI:FinalProject/train/10.edf
Filter and downsample...
ICA artifact removal...
Selecting all EEG Channels...
Got events...
(30, 1126)


...

 86%|████████▌ | 12/14 [1:34:41<17:03, 511.78s/it]


/content/drive/Shareddrives/BCI:FinalProject/train/8.edf
Filter and downsample...
ICA artifact removal...
Selecting all EEG Channels...
Got events...
(30, 1126)


 93%|█████████▎| 13/14 [1:40:51<07:48, 468.74s/it]


/content/drive/Shareddrives/BCI:FinalProject/train/9.edf
Filter and downsample...
ICA artifact removal...
Selecting all EEG Channels...
Got events...
(30, 1126)


100%|██████████| 14/14 [1:48:50<00:00, 466.47s/it]


In [ ]:
n_components = 30
kernels = ['linear', 'poly', 'rbf', 'sigmoid', 'cosine']
for kernel in kernels:
  kpca = KernelPCA(n_components=n_components, kernel=kernel)
  ss = StandardScaler()
  input_dir = TEST_DATA_PATH
  output_dir = KPCA_DATA_PATH + kernel + '/test/'
  os.makedirs(output_dir, exist_ok = True)
  process_directory_memory_efficient(input_dir, output_dir, standardizer=ss, dim_reducer=kpca, n_components=30)

Found 14 available EDF files


  0%|          | 0/14 [00:00<?, ?it/s]


/content/drive/Shareddrives/BCI:FinalProject/test/11.edf
Filter and downsample...
ICA artifact removal...
Selecting all EEG Channels...
Got events...
(30, 1126)


  7%|▋         | 1/14 [01:27<18:59, 87.68s/it]


/content/drive/Shareddrives/BCI:FinalProject/test/1.edf
Filter and downsample...
ICA artifact removal...
Selecting all EEG Channels...
Got events...


...


/content/drive/Shareddrives/BCI:FinalProject/test/7.edf
Filter and downsample...
ICA artifact removal...
Selecting all EEG Channels...
Got events...


 93%|█████████▎| 13/14 [17:37<01:21, 81.24s/it]

(30, 1126)

/content/drive/Shareddrives/BCI:FinalProject/test/8.edf
Filter and downsample...
ICA artifact removal...
Selecting all EEG Channels...
Got events...


100%|██████████| 14/14 [19:00<00:00, 81.49s/it]

(30, 1126)


Create groups of channels based on the lobes of the brain

In [ ]:
lobes = [
    ('frontal', 'F', FRONTAL_DATA_PATH),
    ('temporal', 'T', TEMPORAL_DATA_PATH),
    ('parietal', 'P', PARIETAL_DATA_PATH),
    ('occipital', 'O', OCCIPITAL_DATA_PATH)
]
input_dir = TRAIN_DATA_PATH

for name, key, path in lobes:
  output_dir = path + 'train/'
  os.makedirs(output_dir, exist_ok = True)
  ss = StandardScaler()
  process_directory(input_dir, output_dir, standardizer=ss, selected_channels=key)

Found 14 available EDF files


  0%|          | 0/14 [00:00<?, ?it/s]


/content/drive/Shareddrives/BCI:FinalProject/train/1.edf
Filter and downsample...
ICA artifact removal...
Selecting all EEG Channels...
Refining to only F channels...
Found these channels: ['Fp1', 'Fp2', 'Fpz', 'F7', 'F3', 'Fz', 'F4', 'F8', 'FC5', 'FC1', 'FC2', 'FC6', 'AF7', 'AF3', 'AF4', 'AF8', 'F5', 'F1', 'F2', 'F6', 'FC3', 'FCz', 'FC4', 'FT7', 'FT8', 'FT9', 'FT10', 'AFF1', 'AFz', 'AFF2', 'FFC5h', 'FFC3h', 'FFC4h', 'FFC6h', 'FCC5h', 'FCC3h', 'FCC4h', 'FCC6h', 'AFp3h', 'AFp4h', 'AFF5h', 'AFF6h', 'FFT7h', 'FFC1h', 'FFC2h', 'FFT8h', 'FTT9h', 'FTT7h', 'FCC1h', 'FCC2h', 'FTT8h', 'FTT10h']
Got events...
Standardizing...


  7%|▋         | 1/14 [00:58<12:34, 58.07s/it]


/content/drive/Shareddrives/BCI:FinalProject/train/10.edf
Filter and downsample...
ICA artifact removal...
Selecting all EEG Channels...
Refining to only F channels...
Found these channels: ['Fp1', 'Fp2', 'Fpz', 'F7', 'F3', 'Fz', 'F4', 'F8', 'FC5', 'FC1', 'FC2', 'FC6', 'AF7', 'AF3', 'AF4', 'AF8', 'F5', 'F1', 'F2', 'F6', 'FC3', 'FCz', 'FC4', 'FT7', 'FT8', 'FT9', 'FT10', 'AFF1', 'AFz', 'AFF2', 'FFC5h', 'FFC3h', 'FFC4h', 'FFC6h', 'FCC5h', 'FCC3h', 'FCC4h', 'FCC6h', 'AFp3h', 'AFp4h', 'AFF5h', 'AFF6h', 'FFT7h', 'FFC1h', 'FFC2h', 'FFT8h', 'FTT9h', 'FTT7h', 'FCC1h', 'FCC2h', 'FTT8h', 'FTT10h']
Got events...
Standardizing...


...

 86%|████████▌ | 12/14 [28:57<05:08, 154.42s/it]


/content/drive/Shareddrives/BCI:FinalProject/train/8.edf
Filter and downsample...
ICA artifact removal...
Selecting all EEG Channels...
Refining to only O channels...
Found these channels: ['POz', 'O1', 'Oz', 'O2', 'PO5', 'PO3', 'PO4', 'PO6', 'PO7', 'PO8', 'PO9', 'PO10', 'PPO1', 'PPO2', 'PPO9h', 'PPO5h', 'PPO6h', 'PPO10h', 'POO9h', 'POO3h', 'POO4h', 'POO10h', 'OI1h', 'OI2h']
Got events...
Standardizing...


 93%|█████████▎| 13/14 [30:48<02:21, 141.35s/it]


/content/drive/Shareddrives/BCI:FinalProject/train/9.edf
Filter and downsample...
ICA artifact removal...
Selecting all EEG Channels...
Refining to only O channels...
Found these channels: ['POz', 'O1', 'Oz', 'O2', 'PO5', 'PO3', 'PO4', 'PO6', 'PO7', 'PO8', 'PO9', 'PO10', 'PPO1', 'PPO2', 'PPO9h', 'PPO5h', 'PPO6h', 'PPO10h', 'POO9h', 'POO3h', 'POO4h', 'POO10h', 'OI1h', 'OI2h']
Got events...
Standardizing...


100%|██████████| 14/14 [33:09<00:00, 142.13s/it]


In [ ]:
lobes = [
    ('frontal', 'F', FRONTAL_DATA_PATH),
    ('temporal', 'T', TEMPORAL_DATA_PATH),
    ('parietal', 'P', PARIETAL_DATA_PATH),
    ('occipital', 'O', OCCIPITAL_DATA_PATH)
]
input_dir = TEST_DATA_PATH

for name, key, path in lobes:
  output_dir = path + 'test/'
  os.makedirs(output_dir, exist_ok = True)
  ss = StandardScaler()
  process_directory(input_dir, output_dir, standardizer=ss, selected_channels=key)

Found 14 available EDF files


  0%|          | 0/14 [00:00<?, ?it/s]


/content/drive/Shareddrives/BCI:FinalProject/test/11.edf
Filter and downsample...
ICA artifact removal...
Selecting all EEG Channels...
Refining to only F channels...
Found these channels: ['Fp1', 'Fp2', 'Fpz', 'F7', 'F3', 'Fz', 'F4', 'F8', 'FC5', 'FC1', 'FC2', 'FC6', 'AF7', 'AF3', 'AF4', 'AF8', 'F5', 'F1', 'F2', 'F6', 'FC3', 'FCz', 'FC4', 'FT7', 'FT8', 'FT9', 'FT10', 'AFF1', 'AFz', 'AFF2', 'FFC5h', 'FFC3h', 'FFC4h', 'FFC6h', 'FCC5h', 'FCC3h', 'FCC4h', 'FCC6h', 'AFp3h', 'AFp4h', 'AFF5h', 'AFF6h', 'FFT7h', 'FFC1h', 'FFC2h', 'FFT8h', 'FTT9h', 'FTT7h', 'FCC1h', 'FCC2h', 'FTT8h', 'FTT10h']
Got events...
Standardizing...


  7%|▋         | 1/14 [00:41<09:04, 41.92s/it]


/content/drive/Shareddrives/BCI:FinalProject/test/1.edf
Filter and downsample...
ICA artifact removal...
Selecting all EEG Channels...
Refining to only F channels...
Found these channels: ['Fp1', 'Fp2', 'Fpz', 'F7', 'F3', 'Fz', 'F4', 'F8', 'FC5', 'FC1', 'FC2', 'FC6', 'AF7', 'AF3', 'AF4', 'AF8', 'F5', 'F1', 'F2', 'F6', 'FC3', 'FCz', 'FC4', 'FT7', 'FT8', 'FT9', 'FT10', 'AFF1', 'AFz', 'AFF2', 'FFC5h', 'FFC3h', 'FFC4h', 'FFC6h', 'FCC5h', 'FCC3h', 'FCC4h', 'FCC6h', 'AFp3h', 'AFp4h', 'AFF5h', 'AFF6h', 'FFT7h', 'FFC1h', 'FFC2h', 'FFT8h', 'FTT9h', 'FTT7h', 'FCC1h', 'FCC2h', 'FTT8h', 'FTT10h']
Got events...
Standardizing...


...

 86%|████████▌ | 12/14 [07:09<01:11, 35.60s/it]


/content/drive/Shareddrives/BCI:FinalProject/test/7.edf
Filter and downsample...
ICA artifact removal...
Selecting all EEG Channels...
Refining to only O channels...
Found these channels: ['POz', 'O1', 'Oz', 'O2', 'PO5', 'PO3', 'PO4', 'PO6', 'PO7', 'PO8', 'PO9', 'PO10', 'PPO1', 'PPO2', 'PPO9h', 'PPO5h', 'PPO6h', 'PPO10h', 'POO9h', 'POO3h', 'POO4h', 'POO10h', 'OI1h', 'OI2h']
Got events...
Standardizing...


 93%|█████████▎| 13/14 [07:45<00:35, 35.90s/it]


/content/drive/Shareddrives/BCI:FinalProject/test/8.edf
Filter and downsample...
ICA artifact removal...
Selecting all EEG Channels...
Refining to only O channels...
Found these channels: ['POz', 'O1', 'Oz', 'O2', 'PO5', 'PO3', 'PO4', 'PO6', 'PO7', 'PO8', 'PO9', 'PO10', 'PPO1', 'PPO2', 'PPO9h', 'PPO5h', 'PPO6h', 'PPO10h', 'POO9h', 'POO3h', 'POO4h', 'POO10h', 'OI1h', 'OI2h']
Got events...
Standardizing...


100%|██████████| 14/14 [08:22<00:00, 35.93s/it]


In [ ]:
input_dir = TRAIN_DATA_PATH

output_dir = BASELINE_DATA_PATH + 'train/'
os.makedirs(output_dir, exist_ok = True)
ss = StandardScaler()
process_directory(input_dir, output_dir, standardizer=ss)

Found 14 available EDF files


  0%|          | 0/14 [00:00<?, ?it/s]


/content/drive/Shareddrives/BCI:FinalProject/train/1.edf
Filter and downsample...
ICA artifact removal...
Selecting all EEG Channels...
Got events...
Standardizing...


  7%|▋         | 1/14 [01:12<15:45, 72.73s/it]


/content/drive/Shareddrives/BCI:FinalProject/train/10.edf
Filter and downsample...
ICA artifact removal...
Selecting all EEG Channels...
Got events...
Standardizing...


...

 86%|████████▌ | 12/14 [36:39<06:36, 198.38s/it]


/content/drive/Shareddrives/BCI:FinalProject/train/8.edf
Filter and downsample...
ICA artifact removal...
Selecting all EEG Channels...
Got events...


 93%|█████████▎| 13/14 [39:02<03:01, 181.58s/it]

Standardizing...

/content/drive/Shareddrives/BCI:FinalProject/train/9.edf
Filter and downsample...
ICA artifact removal...
Selecting all EEG Channels...
Got events...
Standardizing...


100%|██████████| 14/14 [42:01<00:00, 180.13s/it]


In [ ]:
input_dir = TEST_DATA_PATH

output_dir = BASELINE_DATA_PATH + 'test/'
os.makedirs(output_dir, exist_ok = True)
ss = StandardScaler()
process_directory(input_dir, output_dir, standardizer=ss)

Found 14 available EDF files


  0%|          | 0/14 [00:00<?, ?it/s]


/content/drive/Shareddrives/BCI:FinalProject/test/11.edf
Filter and downsample...
ICA artifact removal...
Selecting all EEG Channels...
Got events...
Standardizing...


  7%|▋         | 1/14 [00:52<11:17, 52.10s/it]


/content/drive/Shareddrives/BCI:FinalProject/test/1.edf
Filter and downsample...
ICA artifact removal...
Selecting all EEG Channels...
Got events...
Standardizing...


...

 86%|████████▌ | 12/14 [08:00<01:16, 38.36s/it]


/content/drive/Shareddrives/BCI:FinalProject/test/7.edf
Filter and downsample...
ICA artifact removal...
Selecting all EEG Channels...
Got events...
Standardizing...


 93%|█████████▎| 13/14 [08:40<00:38, 38.84s/it]


/content/drive/Shareddrives/BCI:FinalProject/test/8.edf
Filter and downsample...
ICA artifact removal...
Selecting all EEG Channels...
Got events...
Standardizing...


100%|██████████| 14/14 [09:19<00:00, 40.00s/it]


Get channel names and indices

In [15]:
train_files = glob.glob(f'{TRAIN_DATA_PATH}/*.edf') # Must download for yourself ;)
print(f'Found {len(train_files)} available EDF files')
file = train_files[0]
title = os.path.split(file)[-1]
print('\n' + file)
raw = read_raw_edf(file).load_data()
adjust_edf_import(raw)
raw.pick_types(eeg=True)
channels_idx = {}
for i, ch in enumerate(raw.ch_names):
  channels_idx[ch] = i
pickle.dump(channels_idx, open(BASELINE_DATA_PATH +'channel_dict.dat', 'wb'))

print(channels_idx)

Found 14 available EDF files

/content/drive/Shareddrives/BCI:FinalProject/train/1.edf
{'Fp1': 0, 'Fp2': 1, 'Fpz': 2, 'F7': 3, 'F3': 4, 'Fz': 5, 'F4': 6, 'F8': 7, 'FC5': 8, 'FC1': 9, 'FC2': 10, 'FC6': 11, 'M1': 12, 'T7': 13, 'C3': 14, 'Cz': 15, 'C4': 16, 'T8': 17, 'M2': 18, 'CP5': 19, 'CP1': 20, 'CP2': 21, 'CP6': 22, 'P7': 23, 'P3': 24, 'Pz': 25, 'P4': 26, 'P8': 27, 'POz': 28, 'O1': 29, 'Oz': 30, 'O2': 31, 'AF7': 32, 'AF3': 33, 'AF4': 34, 'AF8': 35, 'F5': 36, 'F1': 37, 'F2': 38, 'F6': 39, 'FC3': 40, 'FCz': 41, 'FC4': 42, 'C5': 43, 'C1': 44, 'C2': 45, 'C6': 46, 'CP3': 47, 'CPz': 48, 'CP4': 49, 'P5': 50, 'P1': 51, 'P2': 52, 'P6': 53, 'PO5': 54, 'PO3': 55, 'PO4': 56, 'PO6': 57, 'FT7': 58, 'FT8': 59, 'TP7': 60, 'TP8': 61, 'PO7': 62, 'PO8': 63, 'FT9': 64, 'FT10': 65, 'TPP9h': 66, 'TPP10h': 67, 'PO9': 68, 'PO10': 69, 'P9': 70, 'P10': 71, 'AFF1': 72, 'AFz': 73, 'AFF2': 74, 'FFC5h': 75, 'FFC3h': 76, 'FFC4h': 77, 'FFC6h': 78, 'FCC5h': 79, 'FCC3h': 80, 'FCC4h': 81, 'FCC6h': 82, 'CCP5h': 83, 'CCP